## Building a database with PostgreSQL for crime reports in Boston


In this project, we will use the basic concepts of PostgreSQL to build a database for storing data related with crimes that occurred in Boston. This dataset is available in a file called `boston.csv`. A first output of the first rows is presented here:

| incident_number 	| offense_code 	|      description     	|    date    	| day_of_the_week 	|     lat     	|     long     	|
|:---------------:	|:------------:	|:--------------------:	|:----------:	|:---------------:	|:-----------:	|:------------:	|
| 1               	| 619          	| LARCENY ALL OTHERS   	| 2018-09-02 	| Sunday          	| 42.35779134 	| -71.13937053 	|
| 2               	| 1402         	| VANDALISM            	| 2018-08-21 	| Tuesday         	| 42.30682138 	| -71.06030035 	|
| 3               	| 3410         	| TOWED MOTOR VEHICLE  	| 2018-09-03 	| Monday          	| 42.34658879 	| -71.07242943 	|
| 4               	| 3114         	| INVESTIGATE PROPERTY 	| 2018-09-03 	| Monday          	| 42.33418175 	| -71.07866441 	|

The first column represents the identifier of the crime. The second contains a numeric identifier code for the committed crime. The third represents a description of the crime. The next two rows contain the date on which the crime happened and the corresponding day of the week. Finally, the last two columns represent the location of the crime with a latitude and longitude coordinates.

The main goal of the project is to create a database named `crimes_db` with a table – `boston_crimes` – with appropriate datatypes for storing the data from the `boston.csv` file. The table will be created inside a schema named `crimes`. With respect to the user management, we will create two groups with different privileges to interact with the database: `readonly` and `readwrite`. For both of these groups we will create some fictitious users.

### Step 1: Creating the database

We will start by creating a database for storing our crime data as well as a schema for containing the tables.

In [1]:
import psycopg2

# Connection to the main db
conn = psycopg2.connect("dbname=dq user=dq") # Default database
cursor = conn.cursor()

# Creation of the db
conn.autocommit = True
cursor.execute("CREATE DATABASE crime_db;")
conn.autocommit = False
conn.close() # Closing connection from main db

# Connection to the new db
conn = psycopg2.connect("dbname=crime_db user=dq")
cursor = conn.cursor()

# Creation of the new schema
cursor.execute("CREATE SCHEMA crimes;")

In [ ]:
# Run this cell if deletion of the db, groups or users is needed

# conn.autocommit = True
# cursor.execute("DROP DATABASE IF EXISTS crime_db")
# conn.autocommit = False
# cursor.execute("DROP USER IF EXISTS data_analyst, data_scientist")
# cursor.execute("DROP GROUP IF EXISTS readonly, readwrite")
# conn.commit()

We now have a database and a schema, so we are ready to start creating tables.

### Step 2: Gathering the data

Before to the creation of the new table, let's gather some data about our crime dataset so that we can more easily select the right datatypes to use in our table. Let's start by reading the column names from the `boston.csv` file as well as the first row.

In [2]:
import csv

with open('boston.csv') as file:
    reader = list(csv.reader(file))
    col_headers = reader[0]
    first_row = reader[1]
    
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


We now have access to the column headers and the first row of data. This will help us when we create the table as we will not have to remember the names of the columns and using the first row of data we can easily recall what kind of data is stored in each column.

Before we create a table for storing the crime data, we need to identify the proper datatypes for the columns. To help us with that, let's create a function — `get_col_set()` — that given the name of a CSV file and a column index (starting at 0) that computes a Python set with all distinct values contained in that column. 

This function will be useful for two reasons:

- Checking whether an enumerated datatype might be a good choice for representing a column.
- Computing the maximum length of any text-like column to select appropriate sizes for `VARCHAR` columns.

Columns with a low number of distinct values tend to be good candidates for enumerated datatypes.

In [3]:
def get_col_set(csv_filename, col_index):
    """
    Inputs:
        csv_filename: the name of a CSV file
        col_index: the index of a column of that CSV file
    Output:
        a Python set that contains all distinct values from that column
    """
    import csv
    default_set = set()
    with open(csv_filename) as file:
        next(file) # Skip the header row
        rows = list(csv.reader(file))
        for row in rows:
            default_set.add(row[col_index])
    return default_set

In [4]:
for i in range(len(col_headers)):
    print('Column: {}\t- Nr of different values: {}'.format(col_headers[i], len(get_col_set('boston.csv', i))))

Column: incident_number	- Nr of different values: 298329
Column: offense_code	- Nr of different values: 219
Column: description	- Nr of different values: 239
Column: date	- Nr of different values: 1177
Column: day_of_the_week	- Nr of different values: 7
Column: lat	- Nr of different values: 18177
Column: long	- Nr of different values: 18177


Another important aspect is to know the longest word in any column containing textual data. We can actually use the previous function for computing this:

- Use the `get_col_set()` function to get the distinct values of that column into a variable named `col_values`.
- Iterate over the elements of `col_values` and find the one which is the longest.

There are two textual column in the data set, namely, the `description` and `day_of_the_week` columns.

In [5]:
# Description column
col_values = get_col_set('boston.csv', 2)
max_len = 0
for value in col_values:
    max_len = max(len(value), max_len)
print('Max len for {} column: {}'.format(col_headers[2], max_len))

# Day of the week column
col_values = get_col_set('boston.csv', 4)
max_len = 0
for value in col_values:
    max_len = max(len(value), max_len)
print('Max len for {} column: {}'.format(col_headers[4], max_len))

Max len for description column: 58
Max len for day_of_the_week column: 9


### Step 3: Creating the table and loading the data

Now that we have all the insights we want, it's time to create the table.

In [6]:
# Creation of the enum datatypes
cursor.execute("CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');")

# Creation of the table
cursor.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number integer PRIMARY KEY,
        offense_code integer,
        description varchar(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
);
""")

We have create an enumerated datatype named `weekday` for the `day_of_the_week` column since there there only seven possible values.

For the `incident_number` column we have decided to user the type `integer` and set it as the primary key. The same datatype was also used to represent the `offense_code` column.

Since the `description` column has at most 58 characters we have decided to use the datatype `varchar(100)` for representing it. This leave some margin while not being so big that we will waste a lot of memory.

The `date` was represented as the `date` datatype. Finally, for the latitude and longitude we used `decimal` datatypes to hold quite a lot of precision.

Now that we have created the table, we can load the data into it.

In [7]:
# Loading the data to the table
with open('boston.csv') as file:
    cursor.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", file)

# Testing the data is loaded
cursor.execute("SELECT * FROM crimes.boston_crimes;")
print('Total rows loaded: ', len(cursor.fetchall()))

Total rows loaded:  298329


We have created a database with a schema inside it for hold data about crimes. We selected the right datatypes for storing the data, created a table and loaded the CSV containing crimes about Boston.

### Step 4: User management

Now it is time to handle users. For that, we will create two groups with different privileges: `readonly` and `readwrite`.

By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the public group and on the public schema. Doing this also makes it so that we do not need to revoke the privileges when we create users and groups because unless specified otherwise, privileges are not granted by default.

In [8]:
# Revoke all privileges of the public group
cursor.execute("REVOKE ALL ON SCHEMA public FROM public;")
cursor.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

We have made sure that we are not going to inadvertently inherit privileges from the `public` group. The next step is to create our two users groups.

In [9]:
# Creation of the new groups
cursor.execute("CREATE GROUP readonly NOLOGIN;")
cursor.execute("CREATE GROUP readwrite NOLOGIN;")

# Granting connection privileges
cursor.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cursor.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")

# Granting usage privileges
cursor.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cursor.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")

# Granting specific privileges for each group
cursor.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cursor.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

We have created a readonly group with NOLOGIN because it is a group and not a user. We grant the group the ability to connect to the crime_db and the ability to use the crimes schema.

Then we deal with tables privileges by granting SELECT. We also add an extra line compared with what was asked. This extra line changes the way that privileges are given by default to the readonly group on new table that are created on the crimes schema. As we mentioned, by default not privileges are given. However we change is so that by default any user in the readonly group can issue select commands.

We create a readwrite group with NOLOGIN because it is a group and not a user. We grant the group the ability to connect to the crime_db and the ability to use the crimes schema.

Then we deal with tables privileges by granting SELECT, INSERT, UPDATE and DELETE. As before, we change the default privileges so that user in the readwrite group have these privileges if we ever create a new table on the crimes schema.

With all of this done, the only thing that we need to do is create some users. We are going to create 2 users, one for each group:

- A user named `data_analyst` with password `secret1` in the `readonly` group.

- A user named `data_scientist` with password `secret2` in the `readwrite` group.

In [10]:
# Creation of user 1
cursor.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cursor.execute("GRANT readonly TO data_analyst;")

# Creation of user 2
cursor.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cursor.execute("GRANT readwrite TO data_scientist;")

### Step 5: Testing the database

It is a good practice to test that everything is configured as expected when we finish setting up the database. We can query Postgres internal tables to see this information.

We can use SQL queries to check whether the objects have been created and that users and groups have the right privileges. This requires us to know the Postgres internal tables. We can query the `pg_roles` table to inspect privileges related to the database and the `information_schema.table_privileges` table to inspect table privileges.

In [11]:
# Closing the current connection
conn.commit()
conn.close()

# Connection to the database
conn = psycopg2.connect(dbname="crime_db", user="dq")
cursor = conn.cursor()

# Checking users and groups
cursor.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin
    FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cursor.fetchall():
    print(user)
print()

# Checking privileges
cursor.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cursor.fetchall():
    print(user)

# Closing the connection    
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')


In the `pg_roles` table we can check database related privileges and for that we look at the following columns:

- `rolname`: The name of the user / group that the privilege refers to.
- `rolsuper`: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
- `rolcreaterole`: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
- `rolcreatedb`: Whether user / group can create databases. It should be False on every user / group that we have created.
- `rolcanlogin`: Whether user / group can login. It should be True on the users and False on the groups that we have created.

In the `information_schema.table_privileges` we can check privileges related to SQL queries on tables. We have listed the privileges of each group that we have created.